#### This code is used to obtain descriptors directly from spe calculations and convert into a .csv table

Import from .out file in spe

obtain
electronic
- NBO Charges of donor atom, metal atom
- NBO charge of atoms of different atomic distances
- lone pair occupancy of donor atoms

geometric
- bite angle
- bond distance
- difference in distance
- geometric index
- atom distance between each donor atom

polarizability

HOMO-LUMO (electronic)

percent buried volume
-mordred calculations


In [1]:
import os
from pathlib import Path
import re #Import RegEx
import os
import math
import numpy as np
import pandas as pd
from pathlib import Path
import ast

In [2]:
directory = os.getcwd()
directory

'C:\\Users\\George\\Desktop\\Research_UCLA\\CIC\\Computational Work\\Project cluster\\structures\\structures_ligand_id'

In [3]:
spe_directory = directory + '\\combined-spe-ligand-id'
complex_xyz_directory = directory + '\\complex_xyz'

In [4]:
# Aims
# Extract descriptor from output file of spe.
# Extract geometric descriptors (bond angles, bond lengths) and electronic descriptors (nbo charge, HOMO-LUMO gap)
# Input needed: No. for the donor atoms (A and B) as well as metal atom (Pd)

# Bond distance: given cartesian coordinates, find distance between atoms
# Bond angle: given three atoms, find angle




#Input excel table containing metal, atom_a and atom_b name
# Atoms_label.csv obtained through 'opt_out_to_spe_out_ligand_complex code'
atoms_label_df = pd.read_csv(r'atoms_label.csv')   
# atoms_label_df.iloc[11]

In [5]:
atoms_label_df

,Filename,Pd_atom label,Methyl_atom_1 label,Methyl_atom_2 label,Hydrogens_methyl_1_label,Hydrogens_methyl_2_label,Ligand_atom_1 label,Ligand_atom_2 label,Atoms_distance_2 label,Atoms_distance_3 label,Atoms_distance_4 label,Atoms_distance_2_a label,Atoms_distance_3_a label,Atoms_distance_4_a label,Atoms_distance_2_b label,Atoms_distance_3_b label,Atoms_distance_4_b label,Atom label shortest paths
0,ligand-001-n-n-1,2,3,7,"[6, 4, 5]","[10, 9, 8]",23,22,"[1, 13, 15, 17, 18, 19]","[11, 12, 14, 16, 20, 21, 24, 25, 26, 27, 28, 2...",[],"[17, 18, 19]","[32, 33, 1, 20, 21, 28, 29, 30, 31]","[11, 12]","[1, 13, 15]","[11, 12, 14, 16, 19, 24, 25, 26, 27]","[20, 21]","[2, 23, 19, 1, 22]"
1,ligand-002-p-p-1,2,7,3,"[9, 8, 10]","[4, 6, 5]",16,17,"[1, 40, 13, 18, 51, 29]","[41, 42, 11, 12, 14, 15, 19, 20, 52, 53, 30, 31]","[21, 22, 23, 24, 32, 33, 34, 35, 43, 44, 45, 4...","[1, 18, 29]","[11, 12, 13, 19, 20, 30, 31]","[32, 33, 34, 35, 14, 15, 21, 22, 23, 24]","[13, 40, 51]","[1, 41, 42, 14, 15, 52, 53]","[11, 12, 43, 44, 45, 46, 54, 55, 56, 57]","[2, 16, 1, 13, 17]"
2,ligand-003-p-p-2,3,4,8,"[5, 7, 6]","[10, 11, 9]",1,12,"[32, 2, 43, 17, 21, 54]","[33, 34, 13, 14, 15, 44, 45, 18, 19, 55, 22, 2...","[16, 20, 24, 25, 26, 27, 35, 36, 37, 38, 46, 4...","[17, 43, 54]","[44, 45, 15, 18, 19, 55, 56]","[2, 46, 47, 48, 49, 16, 20, 57, 58, 59, 60]","[2, 21, 32]","[33, 34, 13, 14, 15, 22, 23]","[35, 36, 37, 38, 16, 17, 20, 24, 25, 26, 27]","[3, 1, 17, 15, 2, 12]"
3,ligand-004-p-p-3,3,8,4,"[11, 10, 9]","[6, 5, 7]",45,1,"[42, 46, 16, 20, 57, 31]","[32, 2, 33, 43, 44, 14, 47, 48, 17, 18, 21, 22...","[12, 13, 15, 19, 23, 24, 25, 26, 34, 35, 36, 3...","[42, 46, 57]","[2, 43, 44, 47, 48, 58, 59]","[12, 13, 14, 49, 50, 51, 52, 60, 61, 62, 63]","[16, 20, 31]","[32, 33, 14, 17, 18, 21, 22]","[34, 35, 36, 37, 2, 15, 19, 23, 24, 25, 26]","[3, 45, 42, 2, 14, 16, 1]"
4,ligand-005-c-n-1,1,42,46,"[43, 44, 45]","[49, 48, 47]",2,59,"[50, 52, 61, 6]","[3, 4, 7, 51, 55, 56, 25]","[5, 8, 9, 20, 53, 54, 24, 58]","[6, 61]","[3, 4, 7, 25]","[5, 8, 9, 50, 20, 24]","[50, 52]","[51, 55, 56, 25]","[6, 53, 54, 24, 58]","[1, 2, 6, 25, 50, 59]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,ligand-467-p-o-164,65,89,85,"[90, 92, 91]","[86, 88, 87]",1,2,"[40, 56, 3, 4]","[5, 39, 41, 51, 21, 55, 27]","[64, 38, 6, 70, 10, 42, 75, 16, 49, 52, 20, 54...","[4, 40, 56]","[3, 39, 41, 51, 55, 27]","[64, 5, 38, 70, 42, 75, 49, 52, 21, 54, 28, 29]",[3],"[21, 4, 5]","[6, 10, 16, 20, 27]","[65, 1, 4, 3, 2]"
436,ligand-468-p-o-165,29,34,38,"[36, 35, 37]","[41, 39, 40]",1,31,"[2, 3, 4, 5]","[6, 9, 12, 16, 21, 25]","[33, 7, 8, 42, 11, 13, 15, 17, 22, 24, 61, 30]","[2, 4, 5]","[3, 9, 12, 16, 21, 25]","[33, 6, 8, 42, 11, 13, 15, 17, 22, 24, 30]",[3],"[2, 6]","[7, 9, 61]","[29, 1, 2, 3, 31]"
437,ligand-471-p-o-168,10,26,30,"[28, 29, 27]","[33, 31, 32]",1,11,"[51, 2, 3, 34]","[4, 35, 36, 7, 37, 52, 53, 54]","[5, 6, 9, 12, 38, 39, 40, 41, 42, 43, 55, 56, ...","[2, 34, 51]","[3, 36, 35, 37, 7, 52, 53, 54]","[4, 6, 9, 38, 39, 40, 41, 42, 43, 55, 56, 57, ...",[3],"[2, 4]","[5, 7, 12]","[10, 1, 2, 3, 11]"
438,ligand-472-p-o-169,26,31,35,"[33, 32, 34]","[38, 36, 37]",1,28,"[2, 3, 4, 5]","[7, 9, 13, 18, 22, 59]","[6, 39, 8, 10, 12, 14, 19, 21, 58, 27, 61, 30]","[2, 4, 5]","[3, 7, 9, 13, 18, 22]","[6, 39, 8, 27, 10, 12, 14, 19, 21, 59, 30]",[3],"[2, 59]","[7, 58, 61]","[26, 1, 2, 3, 28]"


In [6]:
xyz_match = ['X           Y           Z']
LUMO_match = ['Alpha virt. eigenvalues']
HOMO_match = ['Alpha  occ. eigenvalues']
nbo_match = ['Natural Population Analysis']
mulliken_match = ['Mulliken charges:']

missing_nbo_spe_calc =[]
missing_mulliken_spe_calc = []
# Initiate list for gathering descriptor values
name_list = []
metal_atom_list = []
ligand_atom_a_list = []
ligand_atom_b_list = []
metal_ligand_distance_1_list = []
metal_ligand_distance_2_list = []
bite_angle_list = []
lumo_list = []
homo_list = []
homo_lumo_list = []
nbo_pd_list = []
nbo_a_list = []
nbo_b_list = []
nbo_a_b_list = []
nbo_a_b_avg_list = []
nbo_dist_2_avg_list = []
nbo_dist_3_avg_list = []
nbo_dist_4_avg_list = []
nbo_dist_2_min_list = []
nbo_dist_3_min_list = []
nbo_dist_4_min_list = []
nbo_dist_2_max_list = []
nbo_dist_3_max_list = []
nbo_dist_4_max_list = []
nbo_dist_2_a_avg_list = []
nbo_dist_3_a_avg_list = []
nbo_dist_4_a_avg_list = []
nbo_dist_2_b_avg_list = []
nbo_dist_3_b_avg_list = []
nbo_dist_4_b_avg_list = []
nbo_dist_2_a_b_avg_diff_list = []
nbo_dist_3_a_b_avg_diff_list = []
nbo_dist_4_a_b_avg_diff_list = []
nbo_dist_2_a_min_list = []
nbo_dist_3_a_min_list = []
nbo_dist_4_a_min_list = []
nbo_dist_2_a_max_list = []
nbo_dist_3_a_max_list = []
nbo_dist_4_a_max_list = []
nbo_dist_2_b_min_list = []
nbo_dist_3_b_min_list = []
nbo_dist_4_b_min_list = []
nbo_dist_2_b_max_list = []
nbo_dist_3_b_max_list = []
nbo_dist_4_b_max_list = []

nbo_metal_donor_diff_list = []
nbo_metal_a_diff_list = []
nbo_metal_b_diff_list = []
nbo_2_1_avg_diff_list = []
nbo_3_2_avg_diff_list = []
nbo_4_3_avg_diff_list = []
nbo_2_1_a_avg_diff_list = []
nbo_3_2_a_avg_diff_list = []
nbo_4_3_a_avg_diff_list = []
nbo_2_1_b_avg_diff_list = []
nbo_3_2_b_avg_diff_list = []
nbo_4_3_b_avg_diff_list = []
 

mulliken_pd_list = []
mulliken_a_list = []
mulliken_b_list = []
mulliken_a_b_list = []
mulliken_dist_2_avg_list = []
mulliken_dist_3_avg_list = []
mulliken_dist_4_avg_list = []

angle_deviation_percent_list = []

In [7]:
# Read in .out file
index_count = 0
for subdir,dirs,files in os.walk(spe_directory):                  # Loop over each directory, subdirectory and files
    for file in files:                                      # Loop over each file
        if any([file.endswith('-spe.out')]):                    # If file is a .out file
            filename = os.path.join(subdir, file)       # Return path to file
            name = Path(filename).stem           # Extract filename from the end of path and return as a string
            name = name.replace('-spe','')     #Remove -opt in string to get just the name of ligand 'pd-x-x-x'
            name_list.append(name)
            print(name)

            mylines = []

            with open (filename, 'rt') as myfile:       # Open .out for reading text
                # myfile = myfile.read()                # Read the entire file to a string
                for myline in myfile:                    # For each line, stored as myline,
                    mylines.append(myline)               # add its contents to mylines list.

                    
                for line in mylines:
                    if 'NAtoms' in line:
                        number_list = re.findall('-?\d*\.?\d+',line)            # get number of atoms
                        natoms = int(number_list[0])     



                # Identify row in atom_label_df that matches the name
                matching_row = atoms_label_df.loc[atoms_label_df['Filename'] == name]
                # print(matching_row)
    
                
            
                # Assign atom labels based on dataframe 
                # print(index_count)
                pd_atom = matching_row.loc[index_count]['Pd_atom label']
                ligand_atom_1 = matching_row.loc[index_count]['Ligand_atom_1 label']
                ligand_atom_2 = matching_row.loc[index_count]['Ligand_atom_2 label']
                atoms_distance_2 = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_2 label'])
                atoms_distance_3 = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_3 label'])
                atoms_distance_4 = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_4 label'])
                atoms_distance_2_a = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_2_a label'])
                atoms_distance_3_a = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_3_a label'])                
                atoms_distance_4_a = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_4_a label'])                
                atoms_distance_2_b = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_2_b label'])
                atoms_distance_3_b = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_3_b label'])                
                atoms_distance_4_b = ast.literal_eval(matching_row.loc[index_count]['Atoms_distance_4_b label'])                 
                atom_label_shortest_path = ast.literal_eval(matching_row.loc[index_count]['Atom label shortest paths'])    
                
                index_count = index_count + 1
    #             metal = atoms_label_df.to_numeric(df.loc[row_index, column_name])   #Extract important atom labels for descriptor generation

                for line in mylines:
                    for phrase in xyz_match:                                # iterate through each phrases
                        if phrase in line:                                          # check if phrase is in line

                                                                                    # For loop for generating the XYZ coordinates
                            count = 0
                            xyz = []
                            line_number = mylines.index(line) + 2
                            while count < natoms:
                                count = count + 1
                                xyz.append(mylines[line_number])
                                line_number = line_number + 1


                x_coord = []
                y_coord = []
                z_coord = []
                atom_symbol =[]
                element = {"1":'H', "6":"C", "7": "N", "8":"O", "9":"F", "14": "Si", "15":"P", "16":"S", "17":"Cl", "26":"Fe", "35":"Br", "46":"Pd"}

                # Generate XYZ file in .txt form, then find xyz coordinates for metal, atom_a and atom_b
                for line in xyz:
                    number_list = re.findall('-?\d*\.?\d+',line)
                    atom_number = int(number_list[0])
                    element_number = int(number_list[1])
                    atom_x = float(number_list[3])
                    atom_y = float(number_list[4])
                    atom_z = float(number_list[5])

                    # Make xyz coord into .txt file for percent buried volume
                    x_coord.append(atom_x)
                    y_coord.append(atom_y)
                    z_coord.append(atom_z)
                    atom_symbol.append(element[str(element_number)])
                    name_xyz = name + '_xyz.txt'            

    #                     xyz_df.to_csv(name_xyz, header=False, index=False, sep = " ")

                    # Identify metal, atom_a and atom_b numbers and obtain their respective x,y,z coords
                    if atom_number == pd_atom:
                        metal_x = atom_x
                        metal_y = atom_y
                        metal_z = atom_z
                        if element_number != 46:
                            print('Wrong metal atom number assignment ',filename) # Checking if metal atom was assigned correctly
                    elif atom_number == ligand_atom_1:
                        a_x = atom_x
                        a_y = atom_y
                        a_z = atom_z
                    elif atom_number == ligand_atom_2:
                        b_x = atom_x
                        b_y = atom_y
                        b_z = atom_z

                data = {
                    'Atom': atom_symbol,
                    'X': x_coord,
                    'Y': y_coord,
                    'Z': z_coord
                }

                xyz_df = pd.DataFrame(data)      


                mylines = []

                with open (filename, 'rt') as myfile:       # Open .out for reading text
                    # myfile = myfile.read()                # Read the entire file to a string
                    for myline in myfile:                    # For each line, stored as myline,
                        mylines.append(myline)               # add its contents to mylines list.

                # print(type(matching_row))

                # Find HOMO - LUMO energy gap in SPE file

                LUMO = []
                for phrase in LUMO_match:
                    for line in mylines:
                        if phrase in line:
                            LUMO.append(line)
                number_list = re.findall('-?\d*\.?\d+',LUMO[0])            # get value from line containing LUMO
                LUMO_energy = float(number_list[0])                         # get first value from the line, which represents the LUMO

                HOMO = []
                for phrase in HOMO_match:
                    for line in mylines:
                        if phrase in line:
                            HOMO.append(line)
                number_list = re.findall('-?\d*\.?\d+',HOMO[-1])            # get value from line containing LUMO
                HOMO_energy = float(number_list[-1])                        # get lsat value from line, which represents HOMO

                HOMO_LUMO = round(LUMO_energy - HOMO_energy,4)    
                # print('LUMO: ', LUMO_energy)
                # print('HOMO: ', HOMO_energy)
                # print('HOMO_LUMO: ', HOMO_LUMO)


                # Find NBO charge 

                #Obtain indexes for calculating NBO charge of different atom distances

                for phrase in nbo_match:                                       # iterate through each phrases
                    nbo_match_count = 0
                    for line in mylines:
                        if phrase in line:                                          # check if phrase is in line
                            nbo_match_count = nbo_match_count + 1
                            count = 0
                            nbo_xyz = []
                            line_number = mylines.index(line) + 6
                            while count < natoms:
                                count = count + 1
                                nbo_xyz.append(mylines[line_number])
                                line_number = line_number + 1

                            with open('nbo.txt', 'w') as filehandle:                   # Save nbo_xyz  into a txt file
                                for listitem in nbo_xyz:
                                    filehandle.write(listitem)
                    
                    

                # Extract NBO charge for metal, atom A and atom B, atom at distance 2,3,4
                if nbo_match_count == 0:
                    print('No NBO Charge calculation for ', name)
                    missing_nbo_spe_calc.append(name)
         
                    metal_nbo = 'Missing'
                    a_nbo = 'Missing'
                    nbo_difference = 'Missing'
                    nbo_a_b_avg = 'Missing'
                    nbo_distance_2_avg = 'Missing'
                    nbo_distance_3_avg = 'Missing'
                    nbo_distance_4_avg = 'Missing'
                    nbo_dist_2_a_avg = 'Missing'                  
                    nbo_dist_3_a_avg = 'Missing'  
                    nbo_dist_4_a_avg ='Missing'                   
                    nbo_dist_2_b_avg = 'Missing'           
                    nbo_dist_3_b_avg = 'Missing'
                    nbo_dist_4_b_avg = 'Missing'
                    
                    nbo_dist_2_min = 'Missing'
                    nbo_dist_3_min = 'Missing'               
                    nbo_dist_4_min ='Missing'
                    nbo_dist_2_max = 'Missing'
                    nbo_dist_3_max = 'Missing'                 
                    nbo_dist_4_max = 'Missing' 
                    
                    nbo_dist_2_a_b_avg_diff = 'Missing'             
                    nbo_dist_3_a_b_avg_diff = 'Missing'           
                    nbo_dist_4_a_b_avg_diff = 'Missing'
                    
                    nbo_dist_2_a_min = 'Missing'
                    nbo_dist_3_a_min = 'Missing'              
                    nbo_dist_4_a_min = 'Missing'                   
                    nbo_dist_2_b_min = 'Missing'
                    nbo_dist_3_b_min = 'Missing'              
                    nbo_dist_4_b_min = 'Missing'   
                    nbo_dist_2_a_max = 'Missing'
                    nbo_dist_3_a_max = 'Missing'         
                    nbo_dist_4_a_max = 'Missing'         
                    nbo_dist_2_b_max = 'Missing'
                    nbo_dist_3_b_max = 'Missing'           
                    nbo_dist_4_b_max = 'Missing'  
                    
                    nbo_metal_donor_diff = 'Missing'  
                    nbo_metal_a_diff ='Missing'                               
                    nbo_metal_b_diff = 'Missing'  
                    nbo_2_1_avg_diff = 'Missing'  
                    nbo_3_2_avg_diff = 'Missing'  
                    nbo_4_3_avg_diff = 'Missing'  
                    nbo_2_1_a_avg_diff = 'Missing'  
                    nbo_3_2_a_avg_diff = 'Missing'  
                    nbo_4_3_a_avg_diff = 'Missing'  
                    nbo_2_1_b_avg_diff = 'Missing'  
                    nbo_3_2_b_avg_diff = 'Missing'  
                    nbo_4_3_b_avg_diff = 'Missing'  
                          
                    
                elif nbo_match_count > 0:

                    nbo_distance_2 = []
                    nbo_distance_3 = []
                    nbo_distance_4 = []
                    nbo_distance_2_a = []
                    nbo_distance_3_a = []
                    nbo_distance_4_a = []   
                    nbo_distance_2_b = []
                    nbo_distance_3_b = []
                    nbo_distance_4_b = []   
                    atom_number_check_3_a = []
                    atom_number_check_4_a = []
                    atom_number_check_3_b = []
                    atom_number_check_4_b = []                    
                    

                    for line in nbo_xyz:
                        number_list = re.findall('-?\d*\.?\d+',line)
                        atom_number = int(number_list[0])

                        if atom_number == pd_atom:
                            metal_nbo = float(number_list[1])
                            print('Pd_nbo: ', metal_nbo)

                        elif atom_number == ligand_atom_1:
                            a_nbo = float(number_list[1])
                            # print('Ligand atom A nbo: ', a_nbo)

                        elif atom_number == ligand_atom_2:
                            b_nbo = float(number_list[1]) 
                            # print('Ligand atom B nbo: ', b_nbo)

                        elif atom_number in atoms_distance_2:
                            nbo_distance_2.append(float(number_list[1]))

                        elif atom_number in atoms_distance_3:
                            nbo_distance_3.append(float(number_list[1]))                

                        elif atom_number in atoms_distance_4:
                            nbo_distance_4.append(float(number_list[1]))
                        
                        if atom_number in atoms_distance_2_a:
                            nbo_distance_2_a.append(float(number_list[1]))

                        elif atom_number in atoms_distance_3_a:
                            nbo_distance_3_a.append(float(number_list[1]))
                            atom_number_check_3_a.append(atom_number)
                            
                        elif atom_number in atoms_distance_4_a:
                            nbo_distance_4_a.append(float(number_list[1]))
                            atom_number_check_4_a.append(atom_number)
                            
                        if atom_number in atoms_distance_2_b:
                            nbo_distance_2_b.append(float(number_list[1]))

                        elif atom_number in atoms_distance_3_b:
                            nbo_distance_3_b.append(float(number_list[1]))
                            atom_number_check_3_b.append(atom_number)
                            
                        elif atom_number in atoms_distance_4_b:
                            nbo_distance_4_b.append(float(number_list[1]))
                            atom_number_check_4_b.append(atom_number)
                                                   
                    nbo_a_b_avg = round((a_nbo+b_nbo)/ 2, 4)
                    nbo_distance_2_avg = round(sum(nbo_distance_2) / len(nbo_distance_2), 4)
                    nbo_distance_3_avg = round(sum(nbo_distance_3) / len(nbo_distance_3),4)                      
                    nbo_a_b_avg = round((a_nbo+b_nbo)/ 2, 4)
                    nbo_distance_2_avg = round(sum(nbo_distance_2) / len(nbo_distance_2), 4)
                    nbo_distance_3_avg = round(sum(nbo_distance_3) / len(nbo_distance_3),4)                       
                    nbo_dist_2_a_avg = round(sum(nbo_distance_2_a) / len(nbo_distance_2_a),4)                    
                    nbo_dist_3_a_avg = round(sum(nbo_distance_3_a) / len(nbo_distance_3_a),4)                                         
                    nbo_dist_2_b_avg = round(sum(nbo_distance_2_b) / len(nbo_distance_2_b),4)                    
                    nbo_dist_3_b_avg = round(sum(nbo_distance_3_b) / len(nbo_distance_3_b),4)     
                    nbo_dist_2_min = min(nbo_distance_2)
                    nbo_dist_3_min = min(nbo_distance_3)                                            
                    nbo_dist_2_max = max(nbo_distance_2)
                    nbo_dist_3_max = max(nbo_distance_3)                    
                    nbo_dist_2_a_b_avg_diff = round(abs(nbo_dist_2_a_avg-nbo_dist_2_b_avg),4)                  
                    nbo_dist_3_a_b_avg_diff = round(abs(nbo_dist_3_a_avg-nbo_dist_3_b_avg),4)                        
                    nbo_dist_2_a_min = min(nbo_distance_2_a)
                    nbo_dist_3_a_min = min(nbo_distance_3_a)                                                        
                    nbo_dist_2_b_min = min(nbo_distance_2_b)
                    nbo_dist_3_b_min = min(nbo_distance_3_b)                                                
                    nbo_dist_2_a_max = max(nbo_distance_2_a)
                    nbo_dist_3_a_max = max(nbo_distance_3_a)                                                      
                    nbo_dist_2_b_max = max(nbo_distance_2_b)
                    nbo_dist_3_b_max = max(nbo_distance_3_b)                    
                                            


                    # print('Average NBO for atom at distance 2: ', nbo_distance_2_avg)
                    # print('Average NBO for atom at distance 3: ', nbo_distance_3_avg)

                    # print('Average NBO for atom at distance 2 from A:', nbo_dist_2_a_avg)
                    # print('Average NBO for atom at distance 3 from A:', nbo_dist_3_a_avg)                    

                    # print('Average NBO for atom at distance 2 from B:', nbo_dist_2_b_avg)
                    # print('Average NBO for atom at distance 3 from B:', nbo_dist_3_b_avg)                    
             
                    # print('Average NBO diff for atom at distance 2: ', nbo_dist_2_a_b_avg_diff)
                    # print('Average NBO diff for atom at distance 3: ', nbo_dist_3_a_b_avg_diff)
  


                    # print('Min NBO for atom at distance 2', nbo_dist_2_min)
                    # print('Min NBO for atom at distance 3', nbo_dist_3_min)                    
              
                    # print('Max NBO for atom at distance 2', nbo_dist_2_max)
                    # print('Max NBO for atom at distance 3', nbo_dist_3_max)                    
                                    
                    # print('Min NBO for atom at distance 2 from A:', nbo_dist_2_a_min)                    
                    # print('Min NBO for atom at distance 3 from A:', nbo_dist_3_a_min)                       
                
                    # print('Min NBO for atom at distance 2 from B:', nbo_dist_2_b_min)                    
                    # print('Min NBO for atom at distance 3 from B:', nbo_dist_3_b_min)                       
                                    
                    # print('Max NBO for atom at distance 2 from A:', nbo_dist_2_a_max)                    
                    # print('Max NBO for atom at distance 3 from A:', nbo_dist_3_a_max)                       
                  
                    # print('Max NBO for atom at distance 2 from B:', nbo_dist_2_b_max)                    
                    # print('Max NBO for atom at distance 3 from B:', nbo_dist_3_b_max)                       
                 

                    # print('Atom 3a check: ', atom_number_check_3_a)
                    # print('Atom 4a check: ', atom_number_check_4_a)                    
                    # print('Atom 3b check: ', atom_number_check_3_b)
                    # print('Atom 4b check: ', atom_number_check_4_b)     

                    nbo_difference = round(abs(a_nbo - b_nbo),4)
                    # print('NBO_diff: ', nbo_difference)

                    #Calculate difference in nbo between each distance
                    nbo_metal_donor_diff = abs(metal_nbo - nbo_a_b_avg)
                    nbo_metal_a_diff = abs(metal_nbo - a_nbo)
                    nbo_metal_b_diff = abs(metal_nbo - b_nbo)
                    nbo_2_1_avg_diff = abs(nbo_distance_2_avg - nbo_a_b_avg)
                    nbo_3_2_avg_diff = abs(nbo_distance_3_avg - nbo_distance_2_avg)       
                    nbo_2_1_a_avg_diff = abs(nbo_dist_2_a_avg - a_nbo)
                    nbo_3_2_a_avg_diff = abs(nbo_dist_3_a_avg - nbo_dist_2_a_avg)
                    nbo_2_1_b_avg_diff = abs(nbo_dist_2_b_avg - b_nbo)
                    nbo_3_2_b_avg_diff = abs(nbo_dist_3_b_avg - nbo_dist_2_b_avg)
       
                    # print('Average NBO metal donor atom diff:',nbo_metal_donor_diff)
                    # print('Difference Metal and Atom A', nbo_metal_a_diff)
                    # print('Difference Metal and Atom B', nbo_metal_b_diff)                    
                    # print('Average NBO distance 2 vs 1 diff:',nbo_2_1_avg_diff)                    
                    # print('Average NBO distance 3 vs 2 diff:',nbo_3_2_avg_diff)                        
                    # print('Average NBO distance 2 vs 1 from a diff:',nbo_2_1_a_avg_diff)                    
                    # print('Average NBO distance 3 vs 2 from a diff:',nbo_3_2_a_avg_diff)                                     
                    # print('Average NBO distance 2 vs 1 from b diff:',nbo_2_1_b_avg_diff)                    
                    # print('Average NBO distance 3 vs 2 from b diff:',nbo_3_2_b_avg_diff)                        

                    if len(nbo_distance_4) == 0:
                        print('Ligand has no more than 3 bond distances')
                        nbo_distance_4_avg = None
                        nbo_dist_4_a_avg = None
                        nbo_dist_4_b_avg = None        
                        nbo_dist_4_min = None
                        nbo_dist_4_max = None
                        nbo_dist_4_a_b_avg_diff = None
                        nbo_dist_4_a_min = None
                        nbo_dist_4_a_max = None
                        nbo_dist_4_b_min = None
                        nbo_dist_4_b_max = None
                        nbo_4_3_avg_diff = None
                        nbo_4_3_a_avg_diff = None
                        nbo_4_3_b_avg_diff = None

                    else:
                        nbo_distance_4_avg = round(sum(nbo_distance_4) / len(nbo_distance_4),4)
                        nbo_dist_4_a_avg = round(sum(nbo_distance_4_a) / len(nbo_distance_4_a),4)    
                        nbo_dist_4_b_avg = round(sum(nbo_distance_4_b) / len(nbo_distance_4_b),4)   
                        nbo_dist_4_min = min(nbo_distance_4)  
                        nbo_dist_4_max = max(nbo_distance_4)   
                        nbo_dist_4_a_b_avg_diff = round(abs(nbo_dist_4_a_avg-nbo_dist_4_b_avg),4)       
                        nbo_dist_4_a_min = min(nbo_distance_4_a)        
                        nbo_dist_4_b_min = min(nbo_distance_4_b)      
                        nbo_dist_4_a_max = max(nbo_distance_4_a)        
                        nbo_dist_4_b_max = max(nbo_distance_4_b)    
                        nbo_4_3_avg_diff = abs(nbo_distance_4_avg - nbo_distance_3_avg)                    
                        nbo_4_3_a_avg_diff = abs(nbo_dist_4_a_avg - nbo_dist_3_a_avg)  
                        nbo_4_3_b_avg_diff = abs(nbo_dist_4_b_avg - nbo_dist_3_b_avg)       


                        # print('Average NBO for atom at distance 4: ', nbo_distance_4_avg)
                        # print('Average NBO for atom at distance 4 from A:', nbo_dist_4_a_avg)
                        # print('Average NBO for atom at distance 4 from B:', nbo_dist_4_b_avg)   
                        # print('Min NBO for atom at distance 4', nbo_dist_4_min)                  
                        # print('Max NBO for atom at distance 4', nbo_dist_4_max)                          
                        # print('Average NBO diff for atom at distance 4: ', nbo_dist_4_a_b_avg_diff)  
                        # print('Min NBO for atom at distance 4 from A:', nbo_dist_4_a_min)   
                        # print('Min NBO for atom at distance 4 from B:', nbo_dist_4_b_min) 
                        # print('Max NBO for atom at distance 4 from A:', nbo_dist_4_a_max) 
                        # print('Max NBO for atom at distance 4 from B:', nbo_dist_4_b_max)                        
                        # print('Average NBO distance 4 vs 3 diff:',nbo_4_3_avg_diff)    
                        # print('Average NBO distance 4 vs 3 from a diff:',nbo_4_3_a_avg_diff)
                        # print('Average NBO distance 4 vs 3 from b diff:',nbo_4_3_b_avg_diff) 


                
                #Find Mulliken charge
                for phrase in mulliken_match:                                       # iterate through each phrases
                    mulliken_match_count = 0
                    for line in mylines:
                        if phrase in line:                                          # check if phrase is in line
                            mulliken_match_count = mulliken_match_count + 1

                            count = 0
                            mulliken_xyz = []
                            line_number = mylines.index(line) + 2
                            while count < natoms:
                                count = count + 1
                                mulliken_xyz.append(mylines[line_number])
                                line_number = line_number + 1

                            with open('mulliken.txt', 'w') as filehandle:                   # Save nbo_xyz  into a txt file
                                for listitem in mulliken_xyz:
                                    filehandle.write(listitem)            

                    
                if mulliken_match_count == 0:
                    print('No Mulliken Charge calculation for ', name)
                    missing_mulliken_spe_calc.append(name)

                    metal_mulliken = None
                    a_mulliken = None
                    b_mulliken = None
                    mulliken_difference = None
                    mulliken_distance_2_avg = None
                    mulliken_distance_3_avg = None
                    mulliken_distance_4_avg = None


                elif mulliken_match_count > 0:




                    # Extract mulliken charge for metal, atom A and atom B
                    mulliken_distance_2 = []
                    mulliken_distance_3 = []
                    mulliken_distance_4 = []
                    for line in mulliken_xyz:
                        number_list = re.findall('-?\d*\.?\d+',line)
                        atom_number = int(number_list[0])

                        if atom_number == pd_atom:
                            metal_mulliken = float(number_list[1])
                            # print('Pd_mulliken: ', metal_mulliken)

                        elif atom_number == ligand_atom_1:
                            a_mulliken = float(number_list[1])
                            # print('Ligand atom A mulliken: ', a_mulliken)

                        elif atom_number == ligand_atom_2:
                            b_mulliken = float(number_list[1]) 
                            # print('Ligand atom B mulliken: ', b_mulliken)

                        elif atom_number in atoms_distance_2:
                            mulliken_distance_2.append(float(number_list[1]))

                        elif atom_number in atoms_distance_3:
                            mulliken_distance_3.append(float(number_list[1]))                

                        elif atom_number in atoms_distance_4:
                            mulliken_distance_4.append(float(number_list[1]))

                    if len(mulliken_distance_2) > 0:
                        mulliken_distance_2_avg = round(sum(mulliken_distance_2) / len(mulliken_distance_2),4)
                    else: mulliken_distance_2_avg = None
                    
                    if len(mulliken_distance_3) > 0:
                        mulliken_distance_3_avg = round(sum(mulliken_distance_3) / len(mulliken_distance_3),4)
                    else: mulliken_distance_3_avg = None

                    if len(mulliken_distance_4) > 0:
                        mulliken_distance_4_avg = round(sum(mulliken_distance_4) / len(mulliken_distance_4),4)
                    else: mulliken_distance_4_avg = None
                        
                    # print('Mulliken for atom at distance 2: ', mulliken_distance_2_avg)
                    # print('Mulliken for atom at distance 3: ', mulliken_distance_3_avg)
                    # print('Mulliken for atom at distance 4: ', mulliken_distance_4_avg)                        


                    mulliken_difference = abs(a_mulliken - b_mulliken)
                    # print('Mulliken_diff: ', mulliken_difference)            

    #           Extract metal - donor atom distance

                metal_atoma_distance = round(math.sqrt((metal_x - a_x) ** 2 + (metal_y - a_y) ** 2 + (metal_z - a_z) ** 2),3)    
                metal_atomb_distance = round(math.sqrt((metal_x - b_x) ** 2 + (metal_y - b_y) ** 2 + (metal_z - b_z) ** 2),3)

                # print('Metal-L Distance_1: ',metal_atoma_distance)
                # print('Metal-L Distance_2: ',metal_atomb_distance)

                metal_atoma_vector = [a_x - metal_x, a_y - metal_y, a_z - metal_z]
                metal_atomb_vector = [b_x - metal_x, b_y - metal_y, b_z - metal_z]

                dot_pdt = np.dot(metal_atoma_vector,metal_atomb_vector)

                #Extract A-M-B angle by using arcos rule
                angle = int(math.degrees(math.acos (dot_pdt / ( metal_atoma_distance * metal_atomb_distance ))))     
                # print('Bite angle: ',angle)      

                
                #Extract total internal angle/len(atoms)
                # calculating sum of interior angle for a polygon with n edges = (n-2)x180 degrees
                
                number_of_sides = len(atom_label_shortest_path)
                sum_of_interior_angle_polygon = (number_of_sides - 2)*180
                
#                 my_list = [1, 2, 3, 4, 5]

                # Number of elements to consider in each iteration
                chunk_size = 3

                # Iterate through the list
                angle_list = []
                for i in range(len(atom_label_shortest_path)):
                    chunk = atom_label_shortest_path[i:i + chunk_size]  
                    if len(chunk) < chunk_size:
                        chunk += atom_label_shortest_path[:chunk_size - len(chunk)]  # Wrap around to the beginning
                    
                    # print(chunk) #chunk is three atom label for each iteration
                    
                    
                    
                    
                    #obtain xyz coordinates for each atom involved based on their label
                    #identify angle with arcos rule
                    #Add angle into a list of angles
                    
                    # print(xyz_df)
                    atom_1_row = xyz_df.loc[chunk[0]-1]
                    atom_2_row = xyz_df.loc[chunk[1]-1]
                    atom_3_row = xyz_df.loc[chunk[2]-1]
#                     print('Atom_1_row: ', atom_1_row)
#                     print('Atom_2_row: ', atom_2_row)
#                     print('Atom_3_row: ', atom_3_row)

                    atom_1_x = atom_1_row['X']
                    atom_1_y = atom_1_row['Y']
                    atom_1_z = atom_1_row['Z']
#                     print('Atom_1_x: ',atom_1_x)
#                     print('Atom_1_y: ',atom_1_y)                
#                     print('Atom_1_z: ',atom_1_z)                

                    atom_2_x = atom_2_row['X']
                    atom_2_y = atom_2_row['Y']
                    atom_2_z = atom_2_row['Z']
#                     print('Atom_2_x: ',atom_2_x)
#                     print('Atom_2_y: ',atom_2_y)                
#                     print('Atom_2_z: ',atom_2_z)   
                    
                    atom_3_x = atom_3_row['X']
                    atom_3_y = atom_3_row['Y']
                    atom_3_z = atom_3_row['Z']
#                     print('Atom_3_x: ',atom_3_x)
#                     print('Atom_3_y: ',atom_3_y)                
#                     print('Atom_3_z: ',atom_3_z)   
                           
                    atom_1_2_distance = round(math.sqrt((atom_1_x - atom_2_x) ** 2 + (atom_1_y - atom_2_y) ** 2 + (atom_1_z - atom_2_z) ** 2),5)  
                    atom_2_3_distance = round(math.sqrt((atom_3_x - atom_2_x) ** 2 + (atom_3_y - atom_2_y) ** 2 + (atom_3_z - atom_2_z) ** 2),5)                    
                   
#                     print('Atom 1-2 Distance: ',atom_1_2_distance)
#                     print('Atom 3-2 Distance: ',atom_2_3_distance)

                    atom_1_2_vector = [atom_1_x - atom_2_x, atom_1_y - atom_2_y, atom_1_z - atom_2_z]
                    atom_2_3_vector = [atom_3_x - atom_2_x, atom_3_y - atom_2_y, atom_3_z - atom_2_z]

                    dot_pdt = np.dot(atom_1_2_vector,atom_2_3_vector)  
                    
                    
                    #Extract A-M-B angle by using arcos rule
                    angle = round(float(math.degrees(math.acos (dot_pdt / ( atom_1_2_distance * atom_2_3_distance)))),5) 
                    angle_list.append(angle)
                    # print('Angle: ',angle)                 
                    # print(f"Iteration {i + 1}: {chunk}")                
                
                total_internal_angle = sum(angle_list)
                # print('Total internal angle: ',total_internal_angle)
                angle_deviation_percent = round(100*abs(total_internal_angle - sum_of_interior_angle_polygon)/sum_of_interior_angle_polygon,4)
                # print('Angle deviation: ',angle_deviation_percent)    


                #Add variables to lists
                
                ligand_atom_a_list.append(ligand_atom_1)
                ligand_atom_b_list.append(ligand_atom_2)
                metal_atom_list.append(pd_atom)
                metal_ligand_distance_1_list.append(metal_atoma_distance)
                metal_ligand_distance_2_list.append(metal_atomb_distance)
                bite_angle_list.append(angle)
                lumo_list.append(LUMO_energy)
                homo_list.append(HOMO_energy)
                homo_lumo_list.append(HOMO_LUMO)
                nbo_pd_list.append(metal_nbo)
                nbo_a_list.append(a_nbo)
                nbo_b_list.append(b_nbo)
                nbo_a_b_list.append(nbo_difference)
                nbo_a_b_avg_list.append(nbo_a_b_avg)
                nbo_dist_2_avg_list.append(nbo_distance_2_avg)
                nbo_dist_3_avg_list.append(nbo_distance_3_avg)
                nbo_dist_4_avg_list.append(nbo_distance_4_avg)
                nbo_dist_2_a_avg_list.append(nbo_dist_2_a_avg)
                nbo_dist_3_a_avg_list.append(nbo_dist_3_a_avg)
                nbo_dist_4_a_avg_list.append(nbo_dist_4_a_avg)
                nbo_dist_2_b_avg_list.append(nbo_dist_2_b_avg)
                nbo_dist_3_b_avg_list.append(nbo_dist_3_b_avg)
                nbo_dist_4_b_avg_list.append(nbo_dist_4_b_avg)
                nbo_dist_2_a_b_avg_diff_list.append(nbo_dist_2_a_b_avg_diff)
                nbo_dist_3_a_b_avg_diff_list.append(nbo_dist_3_a_b_avg_diff)
                nbo_dist_4_a_b_avg_diff_list.append(nbo_dist_4_a_b_avg_diff)            
                nbo_dist_2_a_min_list.append(nbo_dist_2_a_min)
                nbo_dist_3_a_min_list.append(nbo_dist_3_a_min)
                nbo_dist_4_a_min_list.append(nbo_dist_4_a_min)
                nbo_dist_2_b_min_list.append(nbo_dist_2_b_min)
                nbo_dist_3_b_min_list.append(nbo_dist_3_b_min)
                nbo_dist_4_b_min_list.append(nbo_dist_4_b_min)
                nbo_dist_2_a_max_list.append(nbo_dist_2_a_max)
                nbo_dist_3_a_max_list.append(nbo_dist_3_a_max)
                nbo_dist_4_a_max_list.append(nbo_dist_4_a_max)
                nbo_dist_2_b_max_list.append(nbo_dist_2_b_max)
                nbo_dist_3_b_max_list.append(nbo_dist_3_b_max)
                nbo_dist_4_b_max_list.append(nbo_dist_4_b_max)                
                nbo_dist_2_min_list.append(nbo_dist_2_min)
                nbo_dist_3_min_list.append(nbo_dist_3_min)                
                nbo_dist_4_min_list.append(nbo_dist_4_min)  
                nbo_dist_2_max_list.append(nbo_dist_2_max)
                nbo_dist_3_max_list.append(nbo_dist_3_max)                
                nbo_dist_4_max_list.append(nbo_dist_4_max)                  
                nbo_metal_donor_diff_list.append(nbo_metal_donor_diff)
                nbo_metal_a_diff_list.append(nbo_metal_a_diff)                
                nbo_metal_b_diff_list.append(nbo_metal_b_diff)
                nbo_2_1_avg_diff_list.append(nbo_2_1_avg_diff)
                nbo_3_2_avg_diff_list.append(nbo_3_2_avg_diff)
                nbo_4_3_avg_diff_list.append(nbo_4_3_avg_diff)
                nbo_2_1_a_avg_diff_list.append(nbo_2_1_a_avg_diff)
                nbo_3_2_a_avg_diff_list.append(nbo_3_2_a_avg_diff)
                nbo_4_3_a_avg_diff_list.append(nbo_4_3_a_avg_diff)
                nbo_2_1_b_avg_diff_list.append(nbo_2_1_b_avg_diff)
                nbo_3_2_b_avg_diff_list.append(nbo_3_2_b_avg_diff)
                nbo_4_3_b_avg_diff_list.append(nbo_4_3_b_avg_diff)                                 
                    
                
                
                mulliken_pd_list.append(metal_mulliken)
                mulliken_a_list.append(a_mulliken)
                mulliken_b_list.append(b_mulliken)
                mulliken_a_b_list.append(mulliken_difference)
                mulliken_dist_2_avg_list.append(mulliken_distance_2_avg)
                mulliken_dist_3_avg_list.append(mulliken_distance_3_avg)
                mulliken_dist_4_avg_list.append(mulliken_distance_4_avg)
                angle_deviation_percent_list.append(angle_deviation_percent)



# Define data dictionary for excel spreadsheet

data = {'Name': name_list,
       'Metal': metal_atom_list, 
       'Atom_A' : ligand_atom_a_list, 
       'Atom_B' : ligand_atom_b_list, 
       'A-M-B Angle' : bite_angle_list, 
       'M-A Distance' : metal_ligand_distance_1_list, 
       'M-B Distance' : metal_ligand_distance_2_list, 
        'Metal_NBO' : nbo_pd_list, 
        'A_NBO' : nbo_a_list, 
        'B_NBO' : nbo_b_list, 
        'HOMO_LUMO' : homo_lumo_list,
        'A-B_NBO' : nbo_a_b_list,
        'NBO-dist-2-avg': nbo_dist_2_avg_list,
        'NBO-dist-3-avg':nbo_dist_3_avg_list,
        'NBO-dist-4-avg':nbo_dist_4_avg_list,
        'NBO-dist-2-a-avg': nbo_dist_2_a_avg_list,
        'NBO-dist-3-a-avg': nbo_dist_3_a_avg_list,        
        'NBO-dist-4-a-avg': nbo_dist_4_a_avg_list,        
        'NBO-dist-2-b-avg': nbo_dist_2_b_avg_list,
        'NBO-dist-3-b-avg': nbo_dist_3_b_avg_list,        
        'NBO-dist-4-b-avg': nbo_dist_4_b_avg_list,    
        'NBO-dist-2-a-b-diff' : nbo_dist_2_a_b_avg_diff_list,
        'NBO-dist-3-a-b-diff' : nbo_dist_3_a_b_avg_diff_list,        
        'NBO-dist-4-a-b-diff' : nbo_dist_4_a_b_avg_diff_list,    
        'NBO-dist-2-min' : nbo_dist_2_min_list,        
        'NBO-dist-3-min' : nbo_dist_3_min_list,                
        'NBO-dist-4-min' : nbo_dist_4_min_list,                
        'NBO-dist-2-max' : nbo_dist_2_max_list,        
        'NBO-dist-3-max' : nbo_dist_3_max_list,                
        'NBO-dist-4-max' : nbo_dist_4_max_list,                  
        'NBO-dist-2-a-min' : nbo_dist_2_a_min_list,
        'NBO-dist-3-a-min' : nbo_dist_3_a_min_list,
        'NBO-dist-4-a-min' : nbo_dist_4_a_min_list,
        'NBO-dist-2-b-min' : nbo_dist_2_b_min_list,
        'NBO-dist-3-b-min' : nbo_dist_3_b_min_list,
        'NBO-dist-4-b-min' : nbo_dist_4_b_min_list,        
        'NBO-dist-2-a-max' : nbo_dist_2_a_max_list,
        'NBO-dist-3-a-max' : nbo_dist_3_a_max_list,
        'NBO-dist-4-a-max' : nbo_dist_4_a_max_list,
        'NBO-dist-2-b-max' : nbo_dist_2_b_max_list,
        'NBO-dist-3-b-max' : nbo_dist_3_b_max_list,
        'NBO-dist-4-b-max' : nbo_dist_4_b_max_list,              
        'NBO-metal-donor-diff':nbo_metal_donor_diff_list,
        'NBO-metal-a-diff':nbo_metal_a_diff_list,   
        'NBO-metal-b-diff':nbo_metal_b_diff_list,           
        'NBO-dist-2-1-avg-diff':nbo_2_1_avg_diff_list,
        'NBO-dist-3-2-avg-diff':nbo_3_2_avg_diff_list,        
        'NBO-dist-4-3-avg-diff':nbo_4_3_avg_diff_list,        
        'NBO-dist-2-1-a-avg-diff':nbo_2_1_a_avg_diff_list,
        'NBO-dist-3-2-a-avg-diff':nbo_3_2_a_avg_diff_list,        
        'NBO-dist-4-3-a-avg-diff':nbo_4_3_a_avg_diff_list,          
        'NBO-dist-2-1-b-avg-diff':nbo_2_1_b_avg_diff_list,
        'NBO-dist-3-2-b-avg-diff':nbo_3_2_b_avg_diff_list,        
        'NBO-dist-4-3-b-avg-diff':nbo_4_3_b_avg_diff_list,          
        
        
        'Metal_mullkien': mulliken_pd_list,
        'A_mulliken': mulliken_a_list,
        'B_mulliken' : mulliken_b_list,
        'A-B_mulliken': mulliken_a_b_list,
        'Mulliken-dist-2-avg' : mulliken_dist_2_avg_list,
        'Mulliken-dist-3-avg' : mulliken_dist_3_avg_list,
        'Mulliken-dist-4-avg' : mulliken_dist_4_avg_list,        
        'Total internal angle deviation %' : angle_deviation_percent_list
       }  

print(len(name_list))
print(len(nbo_pd_list))

descriptor_data = pd.DataFrame(data)
csv_filename = 'descriptor_data.csv'
descriptor_data.to_csv(csv_filename,index=False) 

ligand-001-n-n-1
Pd_nbo:  0.26015
Ligand has no more than 3 bond distances
ligand-002-p-p-1
Pd_nbo:  -0.11874
ligand-003-p-p-2
Pd_nbo:  -0.12498
ligand-004-p-p-3
Pd_nbo:  -0.0932
ligand-005-c-n-1
Pd_nbo:  0.17756
ligand-006-c-n-2
Pd_nbo:  0.17159
ligand-007-c-n-3
Pd_nbo:  0.17783
ligand-008-c-n-4
Pd_nbo:  0.1714
ligand-009-c-n-5
Pd_nbo:  0.16232
ligand-010-c-n-6
Pd_nbo:  0.21037
ligand-011-c-n-7
Pd_nbo:  0.15382
ligand-013-p-p-5
Pd_nbo:  -0.08867
ligand-014-p-c-1
Pd_nbo:  -0.03138
ligand-015-p-c-2
Pd_nbo:  -0.01968
ligand-016-p-c-3
Pd_nbo:  -0.02354
ligand-017-p-c-4
Pd_nbo:  -0.03063
ligand-018-p-c-5
Pd_nbo:  -0.03881
ligand-019-p-c-6
Pd_nbo:  -0.07342
ligand-020-p-c-7
Pd_nbo:  -0.07515
ligand-021-p-c-8
Pd_nbo:  -0.07514
ligand-022-p-c-9
Pd_nbo:  -0.07412
ligand-027-p-p-6
Pd_nbo:  -0.04802
ligand-029-p-p-8
Pd_nbo:  -0.08615
ligand-030-p-p-9
Pd_nbo:  -0.11917
ligand-031-p-p-10
Pd_nbo:  -0.11552
ligand-032-p-p-11
Pd_nbo:  -0.1061
ligand-033-p-p-12
Pd_nbo:  -0.07148
ligand-035-p-p-14
Pd_n

In [8]:
descriptor_data.loc[68]


Name                                ligand-092-p-o-13
Metal                                              15
Atom_A                                              1
Atom_B                                              5
A-M-B Angle                                   89.1204
                                          ...        
A-B_mulliken                                 1.045956
Mulliken-dist-2-avg                            0.1646
Mulliken-dist-3-avg                           -0.1506
Mulliken-dist-4-avg                           -0.0763
Total internal angle deviation %               7.9125
Name: 68, Length: 62, dtype: object

#### Obtain steric descriptors  from xyz coordinates in complex_xyz directory, add to existing descriptor data

In [9]:
#Obtain buried volume
from morfeus import ConeAngle, read_xyz, BuriedVolume, SASA, SolidAngle
# Initialize a list to store missing filenames
missing_filenames = []

#initiate percent buried volume descriptor list
percent_buried_vol_list = []
percent_buried_volume_SE_list = []
percent_buried_volume_SW_list = []
percent_buried_volume_NE_list = []
percent_buried_volume_NW_list = []
percent_buried_volume_max_list = []
percent_buried_volume_min_list = []
percent_buried_volume_max_min_list = []
percent_buried_volume_NE_NW_list = []
percent_buried_volume_SE_SW_list = []
percent_buried_volume_NE_SE_list = []
percent_buried_volume_NW_SW_list = []
percent_buried_volume_NE_SW_list = []
percent_buried_volume_SE_NW_list = []
percent_buried_volume_asym_1_list = []
percent_buried_volume_asym_2_list = []
percent_buried_volume_asym_3_list = []
percent_buried_volume_octant_1_list = []
percent_buried_volume_octant_2_list = []
percent_buried_volume_octant_3_list = []
percent_buried_volume_octant_4_list = []
percent_buried_volume_octant_5_list = []
percent_buried_volume_octant_6_list = []
percent_buried_volume_octant_7_list = []
percent_buried_volume_octant_8_list = []


# Examine coordinates from opt files 
# Read in .out file from.opt
directory = os.getcwd()
index_count = 0
for subdir,dirs,files in os.walk(complex_xyz_directory):                  # Loop over each directory, subdirectory and files
    for file in files:                                      # Loop over each file
        if any([file.endswith('-complex_xyz.txt')]):                    # If file is a .out file
            filename = os.path.join(subdir, file)       # Return path to file
            name = Path(filename).stem.replace('-complex_xyz',"")         # Extract filename from the end of path and return as a string
            print(name)
            
            # Identify row in atom_label_df that matches the name
            matching_row = atoms_label_df.loc[atoms_label_df['Filename'] == name]
#             matching_row = atoms_label_df[atoms_label_df['Filename'] == name]
            print(matching_row)

            # Check if the matching_row is empty
            if matching_row.empty:
                # Print out the missing row information
                print(f"Missing row for filename: {name}")
                
                # Save the missing filename
                missing_filenames.append(name)
                
                # Skip to the next iteration (next file)
                continue




            # If it doesn't exist in atom_label_df, skip and move on to the next xyz file

            # Assign atom labels based on dataframe 
#             print(index_count)
            pd_atom = matching_row.loc[index_count]['Pd_atom label']
            ligand_atom_1 = matching_row.loc[index_count]['Ligand_atom_1 label']
            ligand_atom_2 = matching_row.loc[index_count]['Ligand_atom_2 label']   
            methyl_atom_1 = matching_row.loc[index_count]['Methyl_atom_1 label']
            methyl_atom_2 = matching_row.loc[index_count]['Methyl_atom_2 label']
            hydrogens_methyl_atom_1 = ast.literal_eval(matching_row.loc[index_count]['Hydrogens_methyl_1_label'])
            hydrogens_methyl_atom_2 = ast.literal_eval(matching_row.loc[index_count]['Hydrogens_methyl_2_label'])                       
            
            
            atoms_to_exclude = []
            atoms_to_exclude.append(methyl_atom_1)
            atoms_to_exclude.append(methyl_atom_2)
            atoms_to_exclude.extend(hydrogens_methyl_atom_1)
            atoms_to_exclude.extend(hydrogens_methyl_atom_2)
            
            
            xz_plane_atoms = []
            xz_plane_atoms.append(methyl_atom_1)
            
            z_axis_atoms = []
            z_axis_atoms.append(methyl_atom_1)
            z_axis_atoms.append(methyl_atom_2)            


#             print('Pd Atom label: ',pd_atom)
#             print('Ligand atom 1 label: ', ligand_atom_1)
#             print('Ligand atom 2 label: ', ligand_atom_2) 
#             print('Methyl_atom_1 label: ', methyl_atom_1)
#             print('Methyl_atom_2 label: ', methyl_atom_2)
#             print('Hydrogens_methyl_1_label: ', hydrogens_methyl_atom_1)
#             print('Hydrogens_methyl_2_label: ', hydrogens_methyl_atom_2)
#             print('Atoms to exclude:', atoms_to_exclude)
#             print('Z axis atoms: ',  z_axis_atoms)
#             print('xz plane atoms: ',xz_plane_atoms)


            elements, coordinates = read_xyz(filename)
            bv = BuriedVolume(elements, coordinates, pd_atom, excluded_atoms=atoms_to_exclude,z_axis_atoms=z_axis_atoms,xz_plane_atoms=xz_plane_atoms)
            percent_buried_vol = bv.fraction_buried_volume*100
            print('Percent Buried Volume: ',percent_buried_vol)
            bv.octant_analysis()
            quadrants = bv.quadrants
            octants = bv.octants
#             print(quadrants) #1-SW, 2-SW, 3-NW,4-NE
#             print(octants)

            # Extract specific values using keys
            percent_buried_volume_SE = quadrants['percent_buried_volume'][1]
            percent_buried_volume_SW = quadrants['percent_buried_volume'][2]
            percent_buried_volume_NW = quadrants['percent_buried_volume'][3]
            percent_buried_volume_NE = quadrants['percent_buried_volume'][4]
            percent_buried_volume_max = max(percent_buried_volume_SE,percent_buried_volume_SW,percent_buried_volume_NW,percent_buried_volume_NE)
            percent_buried_volume_min = min(percent_buried_volume_SE,percent_buried_volume_SW,percent_buried_volume_NW,percent_buried_volume_NE)
            percent_buried_volume_max_min = percent_buried_volume_max - percent_buried_volume_min
            
            # Set hemisphere values (6 combinations)
            percent_buried_volume_NE_NW = (percent_buried_volume_NE + percent_buried_volume_NW)/2
            percent_buried_volume_SE_SW = (percent_buried_volume_SE + percent_buried_volume_SW)/2
            percent_buried_volume_NE_SE = (percent_buried_volume_NE + percent_buried_volume_SE)/2
            percent_buried_volume_NW_SW = (percent_buried_volume_SW + percent_buried_volume_NW)/2
            percent_buried_volume_NE_SW = (percent_buried_volume_NE + percent_buried_volume_SW)/2
            percent_buried_volume_SE_NW = (percent_buried_volume_SE + percent_buried_volume_NW)/2
                
            #Examine asymmetry between hemisphers
            percent_buried_volume_asym_1 = abs(percent_buried_volume_NE_NW-percent_buried_volume_SE_SW)
            percent_buried_volume_asym_2 = abs(percent_buried_volume_NE_SE-percent_buried_volume_NW_SW)
            percent_buried_volume_asym_3 = abs(percent_buried_volume_NE_SW-percent_buried_volume_SE_NW)            
            
            
            #Examine percent buried volume octants
            percent_buried_volume_octant_1 = octants['percent_buried_volume'][0]
            percent_buried_volume_octant_2 = octants['percent_buried_volume'][1]
            percent_buried_volume_octant_3 = octants['percent_buried_volume'][2]
            percent_buried_volume_octant_4 = octants['percent_buried_volume'][3]            
            percent_buried_volume_octant_5 = octants['percent_buried_volume'][4]            
            percent_buried_volume_octant_6 = octants['percent_buried_volume'][5]            
            percent_buried_volume_octant_7 = octants['percent_buried_volume'][6]
            percent_buried_volume_octant_8 = octants['percent_buried_volume'][7]                  
            
            
            print(percent_buried_volume_octant_8)
            
            
            #Append to list
            percent_buried_vol_list.append(percent_buried_vol)
            percent_buried_volume_SE_list.append(percent_buried_volume_SE)
            percent_buried_volume_SW_list.append(percent_buried_volume_SW) 
            percent_buried_volume_NE_list.append(percent_buried_volume_NE) 
            percent_buried_volume_NW_list.append(percent_buried_volume_NW) 
            percent_buried_volume_max_list.append(percent_buried_volume_max) 
            percent_buried_volume_min_list.append(percent_buried_volume_min) 
            percent_buried_volume_max_min_list.append(percent_buried_volume_max_min)
            percent_buried_volume_NE_NW_list.append(percent_buried_volume_NE_NW)
            percent_buried_volume_SE_SW_list.append(percent_buried_volume_SE_SW) 
            percent_buried_volume_NE_SE_list.append(percent_buried_volume_NE_SE) 
            percent_buried_volume_NW_SW_list.append(percent_buried_volume_NW_SW)
            percent_buried_volume_NE_SW_list.append(percent_buried_volume_NE_SW) 
            percent_buried_volume_SE_NW_list.append(percent_buried_volume_SE_NW) 
            percent_buried_volume_asym_1_list.append(percent_buried_volume_asym_1) 
            percent_buried_volume_asym_2_list.append(percent_buried_volume_asym_2) 
            percent_buried_volume_asym_3_list.append(percent_buried_volume_asym_3) 
                      
            percent_buried_volume_octant_1_list.append(percent_buried_volume_octant_1)
            percent_buried_volume_octant_2_list.append(percent_buried_volume_octant_2)
            percent_buried_volume_octant_3_list.append(percent_buried_volume_octant_3)
            percent_buried_volume_octant_4_list.append(percent_buried_volume_octant_4)
            percent_buried_volume_octant_5_list.append(percent_buried_volume_octant_5)
            percent_buried_volume_octant_6_list.append(percent_buried_volume_octant_6)
            percent_buried_volume_octant_7_list.append(percent_buried_volume_octant_7)
            percent_buried_volume_octant_8_list.append(percent_buried_volume_octant_8)
                    
            index_count = index_count + 1

            
# Define data dictionary for excel spreadsheet

data = {'Name': name_list,
       'Metal': metal_atom_list, 
       'Atom_A' : ligand_atom_a_list, 
       'Atom_B' : ligand_atom_b_list, 
       'A-M-B Angle' : bite_angle_list, 
       'M-A Distance' : metal_ligand_distance_1_list, 
       'M-B Distance' : metal_ligand_distance_2_list, 
        'Metal_NBO' : nbo_pd_list, 
        'A_NBO' : nbo_a_list, 
        'B_NBO' : nbo_b_list, 
        'HOMO_LUMO' : homo_lumo_list,
        'A-B_NBO' : nbo_a_b_list,
        'NBO-dist-2-avg': nbo_dist_2_avg_list,
        'NBO-dist-3-avg':nbo_dist_3_avg_list,
        'NBO-dist-4-avg':nbo_dist_4_avg_list,
        'NBO-dist-2-a-avg': nbo_dist_2_a_avg_list,
        'NBO-dist-3-a-avg': nbo_dist_3_a_avg_list,        
        'NBO-dist-4-a-avg': nbo_dist_4_a_avg_list,        
        'NBO-dist-2-b-avg': nbo_dist_2_b_avg_list,
        'NBO-dist-3-b-avg': nbo_dist_3_b_avg_list,        
        'NBO-dist-4-b-avg': nbo_dist_4_b_avg_list,    
        'NBO-dist-2-a-b-diff' : nbo_dist_2_a_b_avg_diff_list,
        'NBO-dist-3-a-b-diff' : nbo_dist_3_a_b_avg_diff_list,        
        'NBO-dist-4-a-b-diff' : nbo_dist_4_a_b_avg_diff_list,    
        'NBO-dist-2-min' : nbo_dist_2_min_list,        
        'NBO-dist-3-min' : nbo_dist_3_min_list,                
        'NBO-dist-4-min' : nbo_dist_4_min_list,                
        'NBO-dist-2-max' : nbo_dist_2_max_list,        
        'NBO-dist-3-max' : nbo_dist_3_max_list,                
        'NBO-dist-4-max' : nbo_dist_4_max_list,                  
        'NBO-dist-2-a-min' : nbo_dist_2_a_min_list,
        'NBO-dist-3-a-min' : nbo_dist_3_a_min_list,
        'NBO-dist-4-a-min' : nbo_dist_4_a_min_list,
        'NBO-dist-2-b-min' : nbo_dist_2_b_min_list,
        'NBO-dist-3-b-min' : nbo_dist_3_b_min_list,
        'NBO-dist-4-b-min' : nbo_dist_4_b_min_list,        
        'NBO-dist-2-a-max' : nbo_dist_2_a_max_list,
        'NBO-dist-3-a-max' : nbo_dist_3_a_max_list,
        'NBO-dist-4-a-max' : nbo_dist_4_a_max_list,
        'NBO-dist-2-b-max' : nbo_dist_2_b_max_list,
        'NBO-dist-3-b-max' : nbo_dist_3_b_max_list,
        'NBO-dist-4-b-max' : nbo_dist_4_b_max_list,              
        'NBO-metal-donor-diff':nbo_metal_donor_diff_list,
        'NBO-metal-a-diff':nbo_metal_a_diff_list,   
        'NBO-metal-b-diff':nbo_metal_b_diff_list,           
        'NBO-dist-2-1-avg-diff':nbo_2_1_avg_diff_list,
        'NBO-dist-3-2-avg-diff':nbo_3_2_avg_diff_list,        
        'NBO-dist-4-3-avg-diff':nbo_4_3_avg_diff_list,        
        'NBO-dist-2-1-a-avg-diff':nbo_2_1_a_avg_diff_list,
        'NBO-dist-3-2-a-avg-diff':nbo_3_2_a_avg_diff_list,        
        'NBO-dist-4-3-a-avg-diff':nbo_4_3_a_avg_diff_list,          
        'NBO-dist-2-1-b-avg-diff':nbo_2_1_b_avg_diff_list,
        'NBO-dist-3-2-b-avg-diff':nbo_3_2_b_avg_diff_list,        
        'NBO-dist-4-3-b-avg-diff':nbo_4_3_b_avg_diff_list,          


        'Metal_mullkien': mulliken_pd_list,
        'A_mulliken': mulliken_a_list,
        'B_mulliken' : mulliken_b_list,
        'A-B_mulliken': mulliken_a_b_list,
        'Mulliken-dist-2-avg' : mulliken_dist_2_avg_list,
        'Mulliken-dist-3-avg' : mulliken_dist_3_avg_list,
        'Mulliken-dist-4-avg' : mulliken_dist_4_avg_list,
        
        'Total internal angle deviation %' : angle_deviation_percent_list,
        
        'Total % VBur' : percent_buried_vol_list,
        '% VBur SE'  : percent_buried_volume_SE_list,
        '% VBur SW'  : percent_buried_volume_SW_list,        
        '% VBur NE'  : percent_buried_volume_NE_list,        
        '% VBur NW'  : percent_buried_volume_NW_list,
        '% VBur max'  : percent_buried_volume_max_list,        
        '% VBur min'  : percent_buried_volume_min_list,         
        '% VBur max min diff'  : percent_buried_volume_max_min_list,   
        '% VBur NE NW diff' : percent_buried_volume_NE_NW_list,
        '% VBur SE SW diff' : percent_buried_volume_SE_SW_list,
        '% VBur NE SE diff' : percent_buried_volume_NE_SE_list,
        '% VBur NW SW diff' : percent_buried_volume_NW_SW_list, 
        '% VBur NE SW diff' : percent_buried_volume_NE_SW_list,         
        '% VBur NW SE diff' : percent_buried_volume_SE_NW_list,         
        '% VBur asymm 1' : percent_buried_volume_asym_1_list,
        '% VBur asymm 2' : percent_buried_volume_asym_2_list,        
        '% VBur asymm 3' : percent_buried_volume_asym_3_list,        
        '% VBur octant 1' : percent_buried_volume_octant_1_list,
        '% VBur octant 2' : percent_buried_volume_octant_2_list,
        '% VBur octant 3' : percent_buried_volume_octant_3_list,
        '% VBur octant 4' : percent_buried_volume_octant_4_list,
        '% VBur octant 5' : percent_buried_volume_octant_5_list,
        '% VBur octant 6' : percent_buried_volume_octant_6_list,
        '% VBur octant 7' : percent_buried_volume_octant_7_list,
        '% VBur octant 8' : percent_buried_volume_octant_8_list,    
               
       }  
# After the loop, you can print or save the list of missing filenames
print("Missing filenames:", missing_filenames)
print(len(name_list))
print(len(percent_buried_volume_octant_8_list))


descriptor_data = pd.DataFrame(data)
csv_filename = 'descriptor_data.csv'
descriptor_data.to_csv(csv_filename,index=False)             
            


ligand-001-n-n-1
           Filename  Pd_atom label  Methyl_atom_1 label  Methyl_atom_2 label  \
0  ligand-001-n-n-1              2                    3                    7   

  Hydrogens_methyl_1_label Hydrogens_methyl_2_label  Ligand_atom_1 label  \
0                [6, 4, 5]               [10, 9, 8]                   23   

   Ligand_atom_2 label   Atoms_distance_2 label  \
0                   22  [1, 13, 15, 17, 18, 19]   

                              Atoms_distance_3 label Atoms_distance_4 label  \
0  [11, 12, 14, 16, 20, 21, 24, 25, 26, 27, 28, 2...                     []   

  Atoms_distance_2_a label             Atoms_distance_3_a label  \
0             [17, 18, 19]  [32, 33, 1, 20, 21, 28, 29, 30, 31]   

  Atoms_distance_4_a label Atoms_distance_2_b label  \
0                 [11, 12]              [1, 13, 15]   

               Atoms_distance_3_b label Atoms_distance_4_b label  \
0  [11, 12, 14, 16, 19, 24, 25, 26, 27]                 [20, 21]   

  Atom label shortest pa

In [10]:
descriptor_data

,Name,Metal,Atom_A,Atom_B,A-M-B Angle,M-A Distance,M-B Distance,Metal_NBO,A_NBO,B_NBO,...,% VBur asymm 2,% VBur asymm 3,% VBur octant 1,% VBur octant 2,% VBur octant 3,% VBur octant 4,% VBur octant 5,% VBur octant 6,% VBur octant 7,% VBur octant 8
0,ligand-001-n-n-1,2,23,22,80.01004,2.324,2.324,0.26015,-0.42686,-0.42686,...,0.067555,0.051249,79.276929,75.605665,79.304883,75.750093,4.034663,0.242266,3.866940,0.228289
1,ligand-002-p-p-1,2,16,17,81.08750,2.404,2.406,-0.11874,1.06691,1.06830,...,5.049152,3.262439,81.564480,92.051808,83.060007,82.575475,5.101565,8.190458,6.671636,0.535781
2,ligand-003-p-p-2,3,1,12,88.85482,2.393,2.398,-0.12498,1.08763,1.08260,...,0.099003,0.173546,74.021618,74.375699,95.387626,96.706113,8.866008,10.035408,10.860045,10.669027
3,ligand-004-p-p-3,3,45,1,95.97953,2.432,2.441,-0.09320,1.06382,1.05890,...,0.822307,1.202013,97.623928,96.230898,70.308423,71.878494,11.675363,12.486023,18.496086,13.054417
4,ligand-005-c-n-1,1,2,59,86.57124,2.143,2.295,0.17756,0.22924,-0.40636,...,15.149786,30.368291,64.340291,88.292024,40.775252,96.850540,44.781960,9.821096,17.815878,11.330600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,ligand-467-p-o-164,65,1,2,79.05611,2.410,2.347,0.17601,1.23805,-1.05913,...,17.178764,0.646431,96.449870,85.645732,50.703504,80.609392,5.744502,0.000000,4.449310,26.709840
436,ligand-468-p-o-165,29,1,31,79.28552,2.409,2.236,0.18785,1.01981,-0.67258,...,11.816297,0.644102,90.933656,86.423779,66.539322,84.411107,7.538204,0.489191,15.234812,33.069325
437,ligand-471-p-o-168,10,1,11,78.50039,2.405,2.209,0.15421,1.03570,-0.69794,...,13.316483,2.120993,72.768356,54.267611,58.674991,84.984160,5.278606,0.712821,0.573053,4.463287
438,ligand-472-p-o-169,26,1,28,79.09274,2.409,2.245,0.18785,1.02488,-0.66367,...,11.889676,1.288204,91.231830,86.591502,63.385203,83.469996,6.708908,0.437943,16.432165,32.994782
